In [1]:
import sys
import os
import math
# import soundfile as sf
import numpy as np
import librosa
from pydub import AudioSegment
import wavio
import copy

In [2]:
sys.path.append("../../")
sys.path.append("../../src/")

In [3]:
from Libs.SharedLibs import getFileList
from SharedLibs.audio_utility import save_wavefile

In [4]:
# target_path = "../CurrentUse/wav_files/Single_Fold/train/positive/moaningtmp/"
# src_wav = "../CurrentUse/wav_files/Single_Fold/train/positive/moaning_56_1.5sec/moaning_01_56.wav"
# wav_name = os.path.basename(src_wav)
# save_path = os.path.join(target_path,wav_name)
# print(f"wav_name:{wav_name}")
# print(f"save_path:{save_path}")

In [5]:
true_train_sample_len = 30225;

In [6]:
def padding(wav_file = None, padlen=30225):
    sound, _ = sf.read(wav_file)
    return np.pad(sound, padlen//2, 'constant')

In [7]:
def padding2(src_wav, dest_wav, padlen):
    pad_ms = padlen*1000  # milliseconds of silence needed
    audio = AudioSegment.from_wav(src_wav)
    silence = AudioSegment.silent(duration=pad_ms)
    silence = silence[:np.abs(len(audio)-len(silence))]
    padded = audio + silence  # Adding silence after the audio
    padded.export(dest_wav, format='wav')

In [8]:
def repeatWavSig(wav_file, dest_path,wantedLenth=5.0,multiplier=4):
    data, samplerate = sf.read(wav_file)
    channels = len(data.shape)
    length_s = len(data)/float(sabmplerate)
    if(length_s < wantedLenth):
        n = math.ceil(multiplier*samplerate/len(data))
    if(channels == 2):
        data = np.tile(data,(n,1))
    else:
        data = np.tile(data,n)
    sf.write(dest_path, data, samplerate)
    # now calculate stft for each channel if stereo

In [9]:
def repeatWavSig2(wav_file, dest_path,repeatTime=4):
    data, samplerate = sf.read(wav_file)
    data = np.tile(data, repeatTime);
    sf.write(dest_path, data, samplerate)

In [10]:
def main_padding_v1():
    # src_wav = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K/full_set/S01_pain_low_01.wav"
    # dest_wav = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_1.5Sec/full_set/S01_pain_low_01.wav"
    # data = padding2(src_wav, dest_wav, 1.5)
    # sf.write(dest_wav, data, 20000)
    src_dir = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K/full_set/"
    wav_lst = getFileList(src_dir);
    save_dir = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_1.5Sec/"
    for wav in wav_lst:
        wavname = os.path.basename(wav)
        save_path = os.path.join(save_dir, wavname)
        print(f"padding {wavname} to 1.5")
        padding2(wav, save_path, 1.5)
        # repeatWavSig(wav, save_path)

In [19]:
def repeat_help_clip_to_end():
    src_dir = "../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s1_processed_clear_clips/"
    raw_wav_lst = getFileList(src_dir);
    save_dir = "../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/"
    clear_clip = "../CurrentUse/mixed_sounds/clear_clips/clear_clip.wav"
    wavfile_fmt = "{}_sigrep.wav"
    # test_wav = "../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s1_processed_clear_clips/man/man_clear_clip_help_001_71.wav"
    clear_wav = wavio.read(clear_clip).data.T[0][:200];
    for f in raw_wav_lst:
        sig = wavio.read(f).data.T[0];
        sig_len = len(sig);
        wav_file = wavfile_fmt.format(os.path.basename(f)[:-4]);
        output_file = os.path.join(save_dir, wav_file);
        while sig_len < true_train_sample_len:
            sig = np.append(sig, sig);
            sig_len = len(sig)
            if sig_len < true_train_sample_len:
                sig = np.append(sig, clear_wav);
        save_wavefile(output_file, sig);
        print(f"processed {output_file}");

In [20]:
repeat_help_clip_to_end()

processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_010_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_002_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_034_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_049_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_026_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_038_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_30225/man_clear_clip_help_045_71_sigrep.wav
processed ../CurrentUse/wav_files/src_wavs/positive/help/help_english_71/s2_extend_to_3022

In [28]:
def peek_wav_length():
    peek_wav_lst = getFileList("../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_1.5Sec/")
    for item in peek_wav_lst:
        sound, sr = sf.read(item)
        print(f"length:{len(sound)}, sr:{sr}-{item}")

In [47]:
def main_repeat():
    # src_wav = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_1.5Sec/full_set/S01_pain_low_01.wav"
    # dest_wav = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_5Sec/full_set/S01_pain_low_01.wav"
    # repeatWavSig2(src_wav, dest_wav)
    src_dir = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_1.5Sec/full_set/"
    wav_lst = getFileList(src_dir);
    save_dir = "../CurrentUse/wav_files/src_wavs/positive/wail_and_moan/vivae_pain_20K_5Sec/full_set/"
    for wav in wav_lst:
        wavname = os.path.basename(wav)
        save_path = os.path.join(save_dir, wavname)
        print(f"repeat {wavname} to 5 sec")
        repeatWavSig2(wav, save_path);

In [48]:
main_repeat()

repeat S04_pain_moderate_03.wav to 5 sec
repeat S02_pain_strong_04.wav to 5 sec
repeat S09_pain_moderate_01.wav to 5 sec
repeat S10_pain_peak_05.wav to 5 sec
repeat S07_pain_low_031.wav to 5 sec
repeat S06_pain_peak_07.wav to 5 sec
repeat S05_pain_moderate_05.wav to 5 sec
repeat S01_pain_low_02.wav to 5 sec
repeat S08_pain_moderate_01.wav to 5 sec
repeat S09_pain_peak_15.wav to 5 sec
repeat S09_pain_low_10.wav to 5 sec
repeat S06_pain_strong_17.wav to 5 sec
repeat S06_pain_low_04.wav to 5 sec
repeat S05_pain_moderate_01.wav to 5 sec
repeat S05_pain_moderate_02.wav to 5 sec
repeat S06_pain_strong_01.wav to 5 sec
repeat S05_pain_strong_05.wav to 5 sec
repeat S11_pain_strong_022.wav to 5 sec
repeat S10_pain_low_01.wav to 5 sec
repeat S02_pain_low_05.wav to 5 sec
repeat S02_pain_peak_07.wav to 5 sec
repeat S09_pain_strong_09.wav to 5 sec
repeat S05_pain_strong_02.wav to 5 sec
repeat S09_pain_peak_13.wav to 5 sec
repeat S11_pain_peak_023.wav to 5 sec
repeat S03_pain_moderate_04.wav to 5 sec